In [ ]:
#installing API Wrapper
# pip install MLB-StatsAPI

In [4]:
#Importing API Wrapper
import statsapi

In [16]:
#Checking homerun leaders for 2022
leaders = statsapi.league_leaders('homeRuns', season = 2022, limit = 5)
leaders = leaders.split('\n')
leaders

['Rank Name                 Team                    Value',
 ' 1   Aaron Judge          New York Yankees         43  ',
 ' 2   Kyle Schwarber       Philadelphia Phillies    34  ',
 ' 3   Yordan Alvarez       Houston Astros           30  ',
 ' 4   Pete Alonso          New York Mets            29  ',
 ' 4   Austin Riley         Atlanta Braves           29  ',
 '']

In [30]:
#Looking at stats for a player
statsapi.player_stat_data(personId = 547180, group="hitting", type = 'season')

{'id': 547180,
 'first_name': 'Bryce',
 'last_name': 'Harper',
 'active': True,
 'current_team': 'Philadelphia Phillies',
 'position': 'RF',
 'nickname': 'Harp',
 'last_played': None,
 'mlb_debut': '2012-04-28',
 'bat_side': 'Left',
 'pitch_hand': 'Right',
 'stats': [{'type': 'season',
   'group': 'hitting',
   'season': '2022',
   'stats': {'gamesPlayed': 64,
    'groundOuts': 57,
    'airOuts': 60,
    'runs': 49,
    'doubles': 21,
    'triples': 1,
    'homeRuns': 15,
    'strikeOuts': 52,
    'baseOnBalls': 26,
    'intentionalWalks': 4,
    'hits': 77,
    'hitByPitch': 3,
    'avg': '.318',
    'atBats': 242,
    'obp': '.385',
    'slg': '.599',
    'ops': '.984',
    'caughtStealing': 2,
    'stolenBases': 9,
    'stolenBasePercentage': '.818',
    'groundIntoDoublePlay': 9,
    'numberOfPitches': 1035,
    'plateAppearances': 275,
    'totalBases': 145,
    'rbi': 48,
    'leftOnBase': 93,
    'sacBunts': 0,
    'sacFlies': 4,
    'babip': '.346',
    'groundOutsToAirouts': '

In [37]:
#Getting player info - will use for player IDs
statsapi.lookup_player('Willson Contreras')

[{'id': 575929,
  'fullName': 'Willson Contreras',
  'firstName': 'Willson',
  'lastName': 'Contreras',
  'primaryNumber': '40',
  'currentTeam': {'id': 112},
  'primaryPosition': {'code': '2', 'abbreviation': 'C'},
  'useName': 'Willson',
  'boxscoreName': 'Contreras, Wn',
  'nickName': 'Venezuela',
  'mlbDebutDate': '2016-06-17',
  'nameFirstLast': 'Willson Contreras',
  'firstLastName': 'Willson Contreras',
  'lastFirstName': 'Contreras, Willson',
  'lastInitName': 'Contreras, W',
  'initLastName': 'W Contreras',
  'fullFMLName': 'Willson Eduardo Contreras',
  'fullLFMName': 'Contreras, Willson Eduardo'}]

In [48]:
#Getting player ID for specific player
player_info = statsapi.lookup_player('Willson Contreras')
player_id = player_info[0]['id']
print(f'Player id is: {player_id}')

Player id is: 575929


In [49]:
#Looking at stats for a player from a fetched player id
statsapi.player_stat_data(personId = player_id, group="hitting", type = 'season')

{'id': 575929,
 'first_name': 'Willson',
 'last_name': 'Contreras',
 'active': True,
 'current_team': 'Chicago Cubs',
 'position': 'C',
 'nickname': 'Venezuela',
 'last_played': None,
 'mlb_debut': '2016-06-17',
 'bat_side': 'Right',
 'pitch_hand': 'Right',
 'stats': [{'type': 'season',
   'group': 'hitting',
   'season': '2022',
   'stats': {'gamesPlayed': 92,
    'groundOuts': 101,
    'airOuts': 67,
    'runs': 54,
    'doubles': 21,
    'triples': 2,
    'homeRuns': 16,
    'strikeOuts': 87,
    'baseOnBalls': 38,
    'intentionalWalks': 0,
    'hits': 85,
    'hitByPitch': 22,
    'avg': '.250',
    'atBats': 340,
    'obp': '.363',
    'slg': '.465',
    'ops': '.828',
    'caughtStealing': 2,
    'stolenBases': 3,
    'stolenBasePercentage': '.600',
    'groundIntoDoublePlay': 11,
    'numberOfPitches': 1536,
    'plateAppearances': 400,
    'totalBases': 158,
    'rbi': 42,
    'leftOnBase': 145,
    'sacBunts': 0,
    'sacFlies': 0,
    'babip': '.291',
    'groundOutsToAirout

In [31]:
#Looking at season dates
statsapi.get("seasons", {"sportId": 1, "season": 2022})

{'copyright': 'Copyright 2022 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'seasons': [{'seasonId': '2022',
   'hasWildcard': True,
   'preSeasonStartDate': '2022-01-01',
   'preSeasonEndDate': '2022-03-16',
   'seasonStartDate': '2022-03-17',
   'springStartDate': '2022-03-17',
   'springEndDate': '2022-04-06',
   'regularSeasonStartDate': '2022-04-07',
   'lastDate1stHalf': '2022-07-17',
   'allStarDate': '2022-07-19',
   'firstDate2ndHalf': '2022-07-21',
   'regularSeasonEndDate': '2022-10-05',
   'postSeasonStartDate': '2022-10-07',
   'postSeasonEndDate': '2022-11-05',
   'seasonEndDate': '2022-11-05',
   'offseasonStartDate': '2022-11-06',
   'offSeasonEndDate': '2022-12-31',
   'seasonLevelGamedayType': 'P',
   'gameLevelGamedayType': 'P',
   'qualifierPlateAppearances': 3.1,
   'qualifierOutsPitched': 3.0}]}

In [ ]:
#Getting playerIDs for a team
